# What is a Plugin?

The Nanome Plugin API provides a way to interface and integrate external software with Nanome’s molecular modeling VR software.

Through this API, users can link up external computational such as molecular dynamics, docking software, and link custom databases.

The extended functionality includes the ability to create new windows inside of the virtual environment and is easily customizable through a drag and drop user interface.

Existing documentation:
https://docs.nanome.ai/plugins/overview.html#nanome-stacks-plugins-and-how-to-use-them


There's two main classes we need to be concerned with right now.
- <code>Plugin</code>: Handles Connections to NTS/ Low level packet stuff
- <code>PluginInstance</code>: Collections of hooks and actions to interact with your Nanome Session


In [ ]:
"""Run this to connect your plugins to to Nanome's DEV NTS server.

Alternatively, host and port can be passed as kwargs into the Plugin.setup() function

To connect your VR Application to the DEV server, please see https://nanome.readthedocs.io/en/latest/#using-plugins 
"""

import nanome

nanome.util.config.set('host', 'example-nts.foobar.com')
nanome.util.config.set('port', 9999)

In [ ]:
"""Get most basic plugin running."""

import nanome
from nanome.api import Plugin, PluginInstance
from nanome.util import Logs

class HelloNanomePlugin(PluginInstance):
    
    def on_run(self):
        message = "Hello Nanome!"
        self.send_notification(nanome.util.enums.NotificationTypes.success, message)
        Logs.message(message)


# Information describing the plugin
name = 'Hello Nanome'
description = "Send a notification that says `Hello Nanome`"
category = 'Demo'
has_advanced = False

# Create Plugin, and attach specific PluginInstance to it.
plugin = Plugin.setup(name, description, category, has_advanced, HelloNanomePlugin)


<hr>

# Available Plugin Actions

Neat! We have a basic plugin, but what can we make it do?

Up to date details can be found here.
https://nanome.readthedocs.io/en/latest/nanome.api.plugin_instance.html

## Summary:
### Event Handlers
- <code>start</code>: Called when user “Activates” the plugin
- <code>update</code>: Called when when instance updates (multiple times per second)
- <code>on_run</code>: Called when user presses "Run"
- <code>on_stop</code>: Called when user disconnects or plugin crashes
- <code>on_advanced_settings</code>: Called when user presses "Advanced Settings"
- <code>on_complex_added</code>: Called whenever a complex is added to the workspace.
- <code>on_complex_removed</code>: Called whenever a complex is removed from the workspace.
- <code>on_presenter_changed</code>: Called when room's presenter changes.       
    
### Spatial Actions
- <code>zoom_on_structures</code>: Repositions and resizes the workspace such that the provided structure(s) will be in the center of the users view.
- <code>center_on_structures</code>: Repositions the workspace such that the provided structure(s) will be in the center of the world.
- <code>request_presenter_info</code>: Requests presenter account info (unique ID, name, email)
- <code>request_controller_transforms</code>: Requests presenter controller info (head position, head rotation, left controller position, left controller rotation, right controller position, right controller rotation)

 
### IO/Streaming

- <code>save_files</code>: Save files on the machine running Nanome, and returns result
- <code>create_writing_stream</code>: Create a stream allowing to continuously update properties of many objects
- <code>create_reading_stream</code>: Create a stream allowing to continuously receive properties of many objects
- <code>open_url</code>: Opens a URL in Nanome's computer browser
- <code>send_files_to_load</code>: Send file(s) to Nanome to load directly using Nanome's importers.
- <code>request_export</code>: Request a file export using Nanome exporters
- <code>set_plugin_list_button</code>: Set text and/or usable state of the buttons on the plugin connection menu in Nanome


### Workspace/Molecule Actions
- <code>request_workspace</code>: Request the entire workspace, in deep mode
- <code>add_to_workspace</code>: Add a list of complexes to the current workspace
- <code>request_complex_list</code>: Request the list of all complexes in the workspace, in shallow mode
- <code>request_complexes</code>: Requests a list of complexes by their indices
- <code>update_workspace</code>:  Replace the current workspace in the scene by the workspace in parameter
- <code>send_notification</code>: Send a notification to the user
- <code>update_structures_deep</code>: Update the specific molecular structures in the scene to match the structures in parameter.
- <code>update_structures_shallow</code>: Update the specific molecular structures in the scene to match the structures in parameter
- <code>apply_color_scheme</code>: Apply a color scheme to selected atoms.
 
### Menus/Stacks
- <code>update_menu</code>:  Update the menu in Nanome
- <code>update_content</code>: Update specific UI elements (button, slider, list...)
- <code>update_node</code>: Updates layout nodes and their children
- <code>set_menu_transform</code>: Update the position, scale, and rotation of the menu
- <code>request_menu_transform</code>: Requests spacial information of the plugin menu (position, rotation, scale)


### Calculations
- <code>add_bonds</code>: Calculate bonds
- <code>add_dssp</code>:  Use DSSP to calculate secondary structures
- <code>add_volume</code>: ???



In [ ]:
# Let's build a plugin that can load a complex from an external file
import requests

import nanome
from nanome.api import Plugin, PluginInstance
from nanome.api.structure import Complex
from nanome.api.ui import Menu
from nanome.util import Logs


class ComplexLoaderPlugin(nanome.PluginInstance):

    def on_run(self):
        rcsb_code = '1TYL'
        new_complex = self.load_complex(rcsb_code)
        self.add_to_workspace([new_complex], self.complex_add_callback)
        
    def load_complex(self, rcsb_code):
        """Retrieve pdb file, and load as Nanome Complex object."""
        Logs.message(f'Loading {rcsb_code} into Workspace')
        url = f'https://files.rcsb.org/view/{rcsb_code}.pdb'
        pdb_string = requests.get(url).content.decode()
        complex = Complex().io.from_pdb(string=pdb_string)
        complex.name = rcsb_code
        return complex  

    def complex_add_callback(self, complex_list):
        # After workspace has been updated, add confirmation to Logs.
        complex = next(iter(complex_list))
        Logs.message(f'{complex.name} load completed')

        
name = 'Complex Loader'
description = "Load a Complex from an external PDB file into your Nanome workspace."
category = 'Demo'
has_advanced = False
plugin = Plugin.setup(name, description, category, has_advanced, ComplexLoaderPlugin)


<hr>

# Adding Menus to your Plugin

Oh yeah, that's looking tasty already! But we can really spice this up with some menus for the user to interact with in VR.

Menus are spatial objects that can be used to provide inputs for an operation. Your standard buttons, sliders, text input, etc.<br>
For details, view our [ReadThedocs](https://nanome.readthedocs.io/en/latest/ui.html)

Menu layouts are ideally designed inside of [Stack Studio](https://docs.nanome.ai/plugins/stackstudio.html). This will export a JSON representation of your menu, which can then be loaded through the Plugin.


![title](assets/complex_loader_menu.png)



In [ ]:
"""Let's create a plugin with a menu containing 1 button, which will load a complex into the workspace."""

import requests

import nanome
from nanome.api import Plugin, PluginInstance
from nanome.api.structure import Complex
from nanome.api.ui import Menu
from nanome.util import Logs


nanome.util.config.set('host', 'plugins.nanome.ai')
nanome.util.config.set('port', 9999)

MENU_PATH = 'assets/complex_loader_menu.json'


class ComplexLoaderPlugin(nanome.PluginInstance):

    # RCSB code that we wish to load into Nanome.
    rcsb_code = '1TYL'
    
    def start(self):
        self.menu = nanome.ui.Menu.io.from_json(MENU_PATH)
        self.configure_menu(self.menu)

    def on_run(self):
        self.menu.enabled = True
        self.update_menu(self.menu)

    def configure_menu(self, menu):
        """Add data to button, and register callbacks."""
        btn = menu.root.find_node('load_btn', True).get_content()
        btn.rcsb_code = self.rcsb_code
        btn.register_pressed_callback(self.load_complex)

    def load_complex(self, btn):
        """Registered as callback functions when buttons are pressed."""
        rcsb_code = btn.rcsb_code
        Logs.message(f'Loading {rcsb_code} into Workspace')
        url = f'https://files.rcsb.org/view/{rcsb_code}.pdb'
        pdb_string = requests.get(url).content.decode()
        new_complex = Complex().io.from_pdb(string=pdb_string)
        new_complex.name = rcsb_code
        self.add_to_workspace([new_complex], self.complex_add_callback)

    def complex_add_callback(self, complex_list):
        # After workspace has been updated, add confirmation to Logs.
        complex = next(iter(complex_list))
        Logs.message(f'{complex.name} load completed')

name = 'Complex Menu Loader'
description = "Press a button to load a Complex from an external PDB file into your Nanome workspace."
category = 'Demo'
has_advanced = False
plugin = Plugin.setup(name, description, category, has_advanced, ComplexLoaderPlugin)

In [ ]:
"""If you need multiple menus, a unique index must be assigned to each."""  

import nanome

class CanOSpamPlugin(nanome.PluginInstance):
    
    menu_count = 5

    def on_run(self):
        # create a large number of menus.
        for i in range(self.menu_count):
            m = nanome.api.ui.Menu()
            m.enabled = True
            m.title = ''
            
            # If this is commented out, only one menu will be created.
            m.index = i

            l = m.root.add_new_label('SPAM')
            l.text_horizontal_align = nanome.util.enums.HorizAlignOptions.Middle
            l.text_vertical_align = nanome.util.enums.VertAlignOptions.Middle
            self.update_menu(m)

def main():
    plugin = nanome.Plugin("Can o' Spam", "Contents may be explosive", "LOL", False)
    plugin.set_plugin_class(CanOSpamPlugin)
    plugin.run()

if __name__ == "__main__":
    main()


# Asyncio Support

A recent update to nanome-lib includes support for Python's asyncio Library. If you are running >= Python 3.7, you can enable asyncio for more Pythonic callback handling

Key Points:
- For asyncio enabled plugins, use <code>nanome.AsyncPluginInstance</code> as the base class for your PluginInstance.

In [ ]:
import nanome

class AsyncPlugin(nanome.AsyncPluginInstance):
    pass

- <code>@async_callback</code> decorator must be used on async functions for internal callbacks (ui callbacks, plugin lifecycle callbacks.) Not needed in async calls called by other async calls. (async in async). See structure prep plugin [prep_structures](https://github.com/nanome-ai/plugin-structure-prep/blob/master/nanome_structure_prep/StructurePrep.py#L36)


In [ ]:
import nanome
from nanome.util import async_callback, Logs

NAME = "Async Test"
DESCRIPTION = "Tests async/await in plugins."
CATEGORY = "testing"
HAS_ADVANCED_OPTIONS = False

class AsyncTest(nanome.AsyncPluginInstance):
    def start(self):
        self.on_run()

    @async_callback
    async def on_run(self):
        shallow = await self.request_complex_list()
        index = shallow[0].index

        deep = await self.request_complexes([index])
        complex = deep[0]
        complex.position.x += 1

        await self.update_structures_deep([complex])
        Logs.message('done')

nanome.Plugin.setup(NAME, DESCRIPTION, CATEGORY, HAS_ADVANCED_OPTIONS, AsyncTest)
